In [1]:
import pandas as pd

import re
import string
import unicodedata

#from cleanco import cleanco

import os

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# Clean SP_Points

In [2]:
%time df_full=pd.read_csv("SceneAnalytics.dbo.SP_Points.csv", encoding='latin-1', parse_dates=['pointdt'])
df_full['ex_transactiondescription'] = df_full['ex_transactiondescription'].astype(str)
df_full.info()

CPU times: user 4.14 s, sys: 333 ms, total: 4.47 s
Wall time: 4.47 s
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1256221 entries, 0 to 1256220
Data columns (total 9 columns):
Unique_member_identifier     1256221 non-null object
pointid                      1256221 non-null int64
pointdt                      1256221 non-null datetime64[ns]
points                       1256221 non-null int64
pointtypeid                  1256221 non-null int64
ex_sourceid                  1250723 non-null float64
TransAmount                  1156176 non-null float64
ex_transactiondescription    1256221 non-null object
MerchantID                   0 non-null float64
dtypes: datetime64[ns](1), float64(3), int64(3), object(2)
memory usage: 86.3+ MB


In [3]:
# Some rows are duplicates; not sure why. But remove them.
df_full = df_full.drop_duplicates()
#df = df_full.sample(n=200000, random_state=1)
df = df_full                              
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1196534 entries, 0 to 1256220
Data columns (total 9 columns):
Unique_member_identifier     1196534 non-null object
pointid                      1196534 non-null int64
pointdt                      1196534 non-null datetime64[ns]
points                       1196534 non-null int64
pointtypeid                  1196534 non-null int64
ex_sourceid                  1191105 non-null float64
TransAmount                  1102150 non-null float64
ex_transactiondescription    1196534 non-null object
MerchantID                   0 non-null float64
dtypes: datetime64[ns](1), float64(3), int64(3), object(2)
memory usage: 91.3+ MB


In [4]:
# How many unique vendors are there? (Surely, we can do better!)
df['ex_transactiondescription'].nunique()

154677

In [5]:
df.groupby(['ex_transactiondescription']).size().sort_values(ascending=False)

ex_transactiondescription
STEP_GOAL_COMPLETE                           42024
ADULT TICKETS-BILLETS POUR ADULTE            20742
INTERVENTION                                 17784
PETRO-CANADA                                 14487
CARA Points Earned                           13155
ADULT TICKETS PREMIUM                        11227
Online bonus en ligne                         9625
SCENE Admit                                   7119
UBER BV                                       4593
Child Tickets                                 4474
CONCESSIONS - POPCORN-MAIS ECLATE             4411
CONCESSIONS - DRINKS-BOISSONS                 3375
REAL CDN SUPERSTORE #1                        3046
APL* ITUNES.COM/BILL                          2757
Child Tickets Premium                         2687
SCENE Card Activation Bonus                   2634
SCENE Premium Admit                           2525
CANADIAN TIRE GAS BAR                         2405
Concession Combo 1                            2222
CIRCL

In [6]:
# OK, this is a big, manual effort to try to clean up as many of the merchant names as possible. Some tests are below to indicate how it works.

def preProcess(column):
    column = column.lower().strip()
    column = re.sub('  +', ' ', column)
    column = re.sub('\n', ' ', column)
    column = re.sub('# *\d+#?', ' ', column)
    column = re.sub('\bq\w\w\b', ' ', column)
    column = re.sub('purchase at', ' ', column)
    column = re.sub('\[merge.*\]\d+', '', column)
    
    column = re.sub('\d+$', ' ', column)
    column = re.sub('inc\.?', ' ', column)
    column = re.sub('\'', '', column)
    column = re.sub('[\(\)]', '', column)
    column = re.sub('`', '', column)
    column = re.sub('\-$', '', column)
    column = re.sub('_$', '', column)
    column = re.sub('#\w{0,2$', '', column)
    
    #ideas: ends in number and maybe 'ca'
    column = re.sub('\d{5,}( ca)?$', '', column)
    
    #column = cleanco(column).clean_name()
    
    column = re.sub("mcdonald ","mcdonalds ", column)
    column = re.sub("bed bath and beyond","bed bath & beyond", column)
    column = re.sub("cdn tire store","canadian tire", column)
    column = re.sub("cdn tire gasbar","cdn tire gas bar", column)
    column = re.sub("7-?eleven","7 eleven", column)
    column = re.sub("h&m","h & m", column)
    column = re.sub("macs conv\.","macs convenience", column)
    column = re.sub("a&w","a & w", column)
    column = re.sub("nofrills","no frills", column)
    column = re.sub("facebk","facebook", column)
    column = re.sub("wal-mart","walmart", column)
    column = re.sub("shoppersdrugmart", "shoppers drug mart", column)
    column = re.sub("rw and c", "rw & c", column)
    column = re.sub("rw *& c", "rw & c", column)
    column = re.sub("real cdn\.? superstore", "real canadian superstore", column)
    column = re.sub("real canadian supersto ", "real canadian superstore ", column)
    column = re.sub("real canadian supersto?", "real canadian superstore", column)
    column = re.sub("petrocan", "petro-canada", column)
    column = re.sub("petro gas", "petro-canada", column)
    column = re.sub("bath and body", "bath & body", column)
    column = re.sub("adult tickets-billets pou", "adult tickets", column)
    column = re.sub("adult ticketsr adulte", "adult tickets", column)
    column = re.sub("air can.*", "air canada", column)
    #column = re.sub("", "", column)

    
    names = ['tim horton', 'macs conv. store',
            "7 eleven","a & w","abercrombie & fitch","air canada","aldo","amazon","apple", "ardene",
              "bar & grill","bath & body works","bed bath & beyond",
              "beer store","bell","best buy","bluenotes","booster juice","boston pizza",
              "browns fine foods","bulk barn","burger king","canadian tire",
              "cdn tire gas bar","chapters","chevron","cineplex",
              "circle k irving","costco gas","costco","couchetard","dairy queen",
              "davids tea","dennys","dollar tree","dollarama","dominos pizza","east side marios",
              "esso","facebook","famous player","fido","food basics","foodland","forever 21",
              "fortinos","freshco","freshii","gap","garage","google","grill & chill","guess",
              "h & m","habitat for humanity","harveys","home depot","hudsons bay",
              "itunes","jean coutu","jugo juice","kami cabs","kfc","la senza","la vie en rose",
              "liquor mart","little caesars pizza","loblaw","longos","lush","m&m food market",
              "macs conv","maries mini mart","marshalls","maxi & cie",
              "mcdonalds","metro","michaels","montanas","national money mart",
              "new york fries","no frills","old navy","papa johns pizza","paypal",
              "petsmart","pharmaprix","pizza hut","pizza nova","pizza pizza",
              "pizzeria","popeyes","presto","rabba fine foods","real cdn superstore",
              "reitmans","rexall","rogers","rona","safeway gas bar","sears","second cup",
              "sephora","shell","shoppers drug mart","sobeys","solo liquor store",
              "sport chek","spotify","st louis bar & grill","staples","starbucks",
              "subway","swiss chalet","t&t","telus","the bargain shop","the body shop",
              "the childrens place","the source","tim hortons","tip top tailors",
              "tommy hilfiger","toys r us","u of a","u of t","uber eats","uber",
              "urban planet","utramar","victorias secret","walmart","wendys","winners",
              "yellow cab","your dollar store","zara","zipcar","zumier", 'petrocan', 'save on foods',
              'pharma plus', 'netflix.com', 'mucho burrito', 'party stuff', 'lcbo', 'circle k', 'home hardware',
              'concessions', 'timeplay', 'jack astors', 'arbys', 'canadas wonderland', 'big bee',
            'canex', 'centennial coll' 'carleton univ', 'chartwells', 'child tickets', 'coca cola', 'dominos', 
             'elmsdale supers', 'fatburger', 'galaxy cinemas', 'georgian college', 'husky',
            'safeway', 'nations fresh food', 'sheridan college', 'villa madina', 'yogen fruz', 'znos',
            'ryanair', 'hakim optical', 'rw & co', 'wine rack', 'hallmark', 'happy dollar mart', 'moores clothing',
            'moneymart', 'needs', 'microsoft', 'rec room', 'real canadian superstore', 'olive garden', 'nordstrom', 'rcss', 
            'red river co-op', 'shoppers drug', 'scene premium admit', 'little caeser', 'ultramar', 'value village',
            'adidas', 'atlantic superstore', 'bath & body', 'beavertails', 'bentley', 'booster juice', 'champs sports', 'chipotle',
            'aeo canada', 'aramark', 'zehrs', 'lawtons drugs', 'landmark', 'kwik way', 'kelseys',
            'journeys', 'jimmy te greek', 'jack and jones']
    for name in names:
        my_regex = r".*" + re.escape(name) + r".*"
        column = re.sub(my_regex, name, column)
        
    provs  = ["ab", "bc", "mb", "nb", "nl", "ns", "nt", "nu", "on", "pe", "qc", "sk", "yt"]
    for prov in provs:
        my_regex = r"\b" + re.escape(prov) + r"\b"
        column = re.sub(my_regex, '', column)
        
    cities  = ["surrey", "brampton", "toronto", 'winnipeg', 'mississauga', 'burnaby', 'ottawa',
               'oshawa', 'whitby', 'london', 'windsor', 'vancouver', 'waterloo', 'surrey',
              'kitchener', 'scarborough', 'saskatoon', 'richmond', 'calgary', 'peterborough', 'edmonton',
              'montreal', 'hamilton', 'quebec city', 'laval', 'halifax', 'markham', 'vaughan', 'gatineau', 'regina', 'richmond hill',
              'richmond', 'oakville', 'burlington', 'sherbrooke', 'saguenay', 'barrie', 'abbotsford', 'guelph', 'kingston', 'ajax', 'niagara falls']
    for city in cities:
        my_regex = r"" + re.escape(city) + r""
        column = re.sub(my_regex, '', column)

    column = re.sub('\d\d\d-\d\d\d-\d\d\d\d', ' ', column)
    column = re.sub('  +', ' ', column)
    column = column.strip().strip('"').strip("'").lower().strip()
    # If data is missing, indicate that by setting the value to `None`
    if not column:
        column = "None"
    return column

In [7]:
preProcess('soda #5434')
preProcess("TIM HORTON'S #5756       EDMONTON     AB")
preProcess("TIM HORTONS #0506# QTH                       2")
preProcess("MB LIQUOR MART #25")
preProcess("WINE RACK 84              [MERGET]6046465725444283")
preProcess("MCDONALD'S #2445         ")
preProcess("MCDONALD #40340          ")

'soda'

'tim horton'

'tim horton'

'liquor mart'

'wine rack'

'mcdonalds'

'mcdonalds'

In [8]:
%time df['ex_transactiondescription'] = df['ex_transactiondescription'].apply(preProcess)

CPU times: user 29min 31s, sys: 3.29 s, total: 29min 34s
Wall time: 29min 34s


In [9]:
df['ex_transactiondescription'].nunique()

# For manuel inspection in Excel
#df.groupby(['ex_transactiondescription_clean']).size().sort_values(ascending=False).to_csv('tmp.csv')

75080

In [10]:
# Always null; drop
df = df.drop(['MerchantID'], axis=1)

df.to_csv('SceneAnalytics.dbo.SP_Points_cleaned.csv', index=False)

# Clean CustomerDetail

In [11]:
%time df=pd.read_csv("SceneAnalytics.dbo.SP_CustomerDetail.csv", encoding='latin-1')
df.info()
df.head()

CPU times: user 72 ms, sys: 2.01 ms, total: 74 ms
Wall time: 120 ms
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18000 entries, 0 to 17999
Data columns (total 8 columns):
Unique_member_identifier    18000 non-null object
City                        18000 non-null object
StateProv                   18000 non-null object
FSA                         18000 non-null object
PointsTotal                 18000 non-null int64
LanguagePreference          18000 non-null object
gender                      18000 non-null object
age_class                   18000 non-null object
dtypes: int64(1), object(7)
memory usage: 1.1+ MB


,Unique_member_identifier,City,StateProv,FSA,PointsTotal,LanguagePreference,gender,age_class
0,3EC07935-F94A-4069-B03C-FCB7909D2226,BRADFORD,ON,L3Z,5284,English,Female,14-17
1,E7ADE72A-7B37-4CE4-8C6B-CBD91C563E59,PRIDDIS,AB,T0L,1617,English,Male,14-17
2,DC64E547-CC11-45A2-A1B7-2F47674DA8DB,Surrey,BC,V4N,1158,English,Female,35-44
3,E186B0AF-3ACC-436E-A5B0-6B999EED8E19,Edmonton,AB,T6L,639,English,Female,18-24
4,610D46DC-934A-4BBA-8975-C67282AC6DE0,VANCOUVER,BC,V5N,0,English,Male,18-24


In [12]:
df['City'].nunique()

3430

In [13]:
import string

# Let's try to clean up some of the city names.

def normalizeCity(column):
    column = column.lower().strip()
    column = re.sub('  +', ' ', column)
    column = column.translate(str.maketrans('', '', string.punctuation))
    column = re.sub(r'missis.*', 'mississauga', column)
    
    column = re.sub(r'\bste\b', 'st', column)
    column = re.sub(r'\bsaint\b', 'st', column)
    column = re.sub(r'\'', '', column)
    column = re.sub(r'-', ' ', column)
    
    provs  = ["ab", "bc", "mb", "nb", "nl", "ns", "nt", "nu", "on", "pe", "qc", "sk", "yt"]
    for prov in provs:
        my_regex = r"\b" + re.escape(prov) + r"\b"
        column = re.sub(my_regex, '', column)
    

        
    names = ['tim horton', 'macs conv. store']
    for name in names:
        my_regex = r".*" + re.escape(name) + r".*"
        column = re.sub(my_regex, name, column)
        
    return column.title()

In [14]:
normalizeCity('St. Johns')
normalizeCity('St Johns')
normalizeCity('Saint Johns')
normalizeCity('Ste. Johns')
normalizeCity('Ste Johns')
normalizeCity('St. John\'s')

'St Johns'

'St Johns'

'St Johns'

'St Johns'

'St Johns'

'St Johns'

In [15]:
%time df['City'] = df['City'].apply(normalizeCity)

CPU times: user 1.67 s, sys: 7 ms, total: 1.68 s
Wall time: 1.68 s


In [16]:
df['City'].nunique()

2222

In [17]:
df.to_csv("SceneAnalytics.dbo.SP_CustomerDetail_cleaned.csv", index=False)